In [37]:
import pandas as pd
import numpy as np
import datetime
import requests
from requests.auth import HTTPBasicAuth
import tarfile
import os
import shutil
import warnings
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

#keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout,Conv2D, Flatten, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model,Sequential 
from tensorflow.keras import optimizers, regularizers 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

In [38]:
def get_organize_files(): 
    #download files with images
    #https://talhassner.github.io/home/projects/Adience/Adience-data.html
    print('starting download')
    BASE_URL = "http://www.cslab.openu.ac.il/download/adiencedb/AdienceBenchmarkOfUnfilteredFacesForGenderAndAgeClassification"
    DOWNLOAD_FILES = {
              "http://www.cslab.openu.ac.il/download/adiencedb/AdienceBenchmarkOfUnfilteredFacesForGenderAndAgeClassification/aligned.tar.gz": "aligned.tar.gz",
              "http://www.cslab.openu.ac.il/download/adiencedb/AdienceBenchmarkOfUnfilteredFacesForGenderAndAgeClassification/fold_0_data.txt": "fold_0_data.txt",
              "http://www.cslab.openu.ac.il/download/adiencedb/AdienceBenchmarkOfUnfilteredFacesForGenderAndAgeClassification/fold_1_data.txt": "fold_1_data.txt",
              "http://www.cslab.openu.ac.il/download/adiencedb/AdienceBenchmarkOfUnfilteredFacesForGenderAndAgeClassification/fold_2_data.txt": "fold_2_data.txt",
              "http://www.cslab.openu.ac.il/download/adiencedb/AdienceBenchmarkOfUnfilteredFacesForGenderAndAgeClassification/fold_3_data.txt": "fold_3_data.txt",
              "http://www.cslab.openu.ac.il/download/adiencedb/AdienceBenchmarkOfUnfilteredFacesForGenderAndAgeClassification/fold_4_data.txt": "fold_4_data.txt"
                    }

    for DOWNLOAD_FILE, FILE_NAME in DOWNLOAD_FILES.items():
        if not os.path.exists(FILE_NAME):
            with open(FILE_NAME, 'wb') as file:
                r = requests.get(DOWNLOAD_FILE, auth = HTTPBasicAuth('adiencedb', 'adience'))
                file.write(r.content)
                print('downloaded {}'.format(FILE_NAME))
    
    print('unziping images...')
    
    if not os.path.exists('aligned'):
        with tarfile.open('aligned.tar.gz') as file:
            file.extractall()
        
    print('images unziped')
    
    #compile all files catalog
    fold = pd.read_csv('fold_0_data.txt',sep='\t')
    fold.rename(columns={'user_id':'user_id'},inplace=True)
    fold['fold'] = 0
    for i in range(1,5):
        temp = pd.read_csv('fold_'+str(i)+'_data.txt',sep='\t')
        temp['fold'] = i
        fold = fold.append(temp,ignore_index=True)

    fold.dropna(subset=['gender'],inplace=True)
    fold = fold[['user_id','original_image','face_id','age','gender','fold']]
     
    train = fold.groupby('gender',as_index=False,group_keys=False).apply(lambda x: x.sample(frac=.6))
    test = fold.drop(train.index).groupby('gender',as_index=False,group_keys=False).apply(lambda x: x.sample(frac=.4))
    validation = fold.drop(train.index).drop(test.index)
    
    print('Train and test split done')
    
    #creates directories for training, test and validation files
    cwd = os.getcwd()
    
    train_faces = os.path.join(cwd,'train')
    if not os.path.exists(train_faces):
        os.mkdir(train_faces)

    test_faces = os.path.join(cwd,'test')
    if not os.path.exists(test_faces):
        os.mkdir(test_faces)
    
    validation_faces = os.path.join(cwd,'validation')
    if not os.path.exists(validation_faces):
        os.mkdir(validation_faces)

    #copy images to each directory
    for index, row in train.iterrows():
        src = os.path.join(cwd,'aligned',row['user_id'],'landmark_aligned_face.'+str(row['face_id'])+'.'+row['original_image'])
        dst = os.path.join(train_faces,row['original_image'])
        shutil.copyfile(src,dst)

    print('train images copied')

    for index, row in test.iterrows():
        src = os.path.join(cwd,'aligned',row['user_id'],'landmark_aligned_face.'+str(row['face_id'])+'.'+row['original_image'])
        dst = os.path.join(test_faces,row['original_image'])
        shutil.copyfile(src,dst)
        
    for index, row in validation.iterrows():
        src = os.path.join(cwd,'aligned',row['user_id'],'landmark_aligned_face.'+str(row['face_id'])+'.'+row['original_image'])
        dst = os.path.join(validation_faces,row['original_image'])
        shutil.copyfile(src,dst)

    print('train images copied')

    print('test images copied')
    
    print('validation images copied')
    
    return cwd,fold,train_faces,test_faces,validation_faces,train,test,validation


In [39]:
cwd,fold,train_faces,test_faces,validation_faces,train_images_ids,test_images_ids,validation_images_ids = get_organize_files()

starting download
unziping images...
images unziped
Train and test split done
train images copied
train images copied
test images copied
validation images copied


In [7]:
def Model():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), input_shape = (150, 150, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(64, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(32, (1, 1), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Flatten())
    model.add(Dense(units = 192, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units = 128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units = 32, activation = 'relu'))
    model.add(Dense(units = 3, activation = 'softmax'))
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer = optimizers.RMSprop(lr=1e-4), metrics=['acc'])
    
    return model

In [40]:
def best_so_far():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape = (150, 150, 3), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(64, (5, 5), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(128, (7, 7), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(256, (7, 7), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Flatten())
    model.add(Dense(units = 256, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(units = 128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units = 64, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(units = 32, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(units = 16, activation = 'relu'))
    model.add(Dense(units = 3, activation = 'softmax'))
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer = optimizers.RMSprop(lr=1e-4), metrics=['acc'])
    
    return model

In [8]:
def Model_gender():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape = (150, 150, 3), activation = 'relu', kernel_regularizer=regularizers.L2(l2= 0.001)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), activation = 'relu', kernel_regularizer=regularizers.L2(l2= 0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    
    model.add(Conv2D(64, (3, 3), activation = 'relu', kernel_regularizer=regularizers.L2(l2= 0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    
    model.add(Conv2D(128, (3, 3), activation = 'relu'), kernel_regularizer=regularizers.L2(l2= 0.001))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), activation = 'relu', kernel_regularizer=regularizers.L2(l2= 0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    
    model.add(Conv2D(256, (3, 3), activation = 'relu'), kernel_regularizer=regularizers.L2(l2= 0.001))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), activation = 'relu'), kernel_regularizer=regularizers.L2(l2= 0.001))
    model.add(BatchNormalization())
     
    model.add(Conv2D(512, (5, 5), activation = 'relu'), kernel_regularizer=regularizers.L2(l2= 0.001))
    model.add(BatchNormalization())
    
    model.add(Flatten())
    
    model.add(Dense(units = 256, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units = 128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units = 32, activation = 'relu'))
    
    model.add(Dense(units = 3, activation = 'softmax'))
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['acc'])
    
    return model

In [9]:
def Model_gender_test():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape = (150, 150, 3), activation = 'relu', kernel_regularizer=regularizers.L2(l2= 0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), activation = 'relu', kernel_regularizer=regularizers.L2(l2= 0.001)))
    model.add(BatchNormalization())

    model.add(Conv2D(64, (3, 3), activation = 'relu', kernel_regularizer=regularizers.L2(l2= 0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    
    model.add(Conv2D(128, (3, 3), activation = 'relu', kernel_regularizer=regularizers.L2(l2= 0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    
    model.add(Conv2D(256, (3, 3), activation = 'relu', kernel_regularizer=regularizers.L2(l2= 0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    
    model.add(Flatten())
    
    model.add(Dense(units = 256, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units = 128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units = 32, activation = 'relu'))
    
    model.add(Dense(units = 3, activation = 'softmax'))
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['acc'])
    
    return model

In [10]:
def Model_gender_test2():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape = (150, 150, 3), activation = 'relu', kernel_regularizer=regularizers.L2(l2= 0.002)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), activation = 'relu', kernel_regularizer=regularizers.L2(l2= 0.002)))
    model.add(BatchNormalization())

    model.add(Conv2D(64, (3, 3), activation = 'relu', kernel_regularizer=regularizers.L2(l2= 0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    
    model.add(Conv2D(128, (3, 3), activation = 'relu', kernel_regularizer=regularizers.L2(l2= 0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    
    model.add(Conv2D(256, (3, 3), activation = 'relu', kernel_regularizer=regularizers.L2(l2= 0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    
    model.add(Flatten())
    
    model.add(Dense(units = 256, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(units = 128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(units = 64, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dense(units = 32, activation = 'relu'))
    model.add(Dense(units = 3, activation = 'softmax'))
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['acc'])
    
    return model

In [11]:
def Model_gender2():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape = (150, 150, 3), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    
    model.add(Conv2D(64, (5, 5), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    
    model.add(Conv2D(128, (7, 7), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    
    model.add(Conv2D(256, (7, 7), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    
    model.add(Flatten())
    
    model.add(Dense(units = 256, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(units = 128, activation = 'relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(units = 64, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(Dense(units = 32, activation = 'relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(units = 16, activation = 'relu'))
    model.add(Dense(units = 3, activation = 'softmax'))
    
    model.summary()
    
    model.compile(loss='categorical_crossentropy', optimizer = optimizers.RMSprop(lr=1e-4), metrics=['acc'])
    
    return model

In [12]:
#vectorizing images 
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
                    train_images_ids,
                    x_col='original_image',
                    y_col='gender',
                    directory=train_faces,
                    target_size=(150,150),
                    batch_size=30,
                    class_mode='categorical',
                   # rotation_range = 40,
                   # width_shift_range = 0.2,
                   # height_shift_range = 0.2,
                   # shear_range = 0.2,
                   # zoom_range = 0.2,
                   # horizontal_flip = True,
                   # fill_mode = 'nearest'
                    ) 


validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
                    validation_images_ids,
                    x_col='original_image',
                    y_col='gender',
                    directory=validation_faces,
                    target_size=(150,150),
                    batch_size=30,
                    class_mode='categorical',
                    )

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
                    test_images_ids,
                    x_col='original_image',
                    y_col='gender',
                    directory=test_faces,
                    target_size=(150,150),
                    batch_size=100,
                    class_mode='categorical',
                    )

Found 11154 validated image filenames belonging to 3 classes.
Found 4462 validated image filenames belonging to 3 classes.
Found 2975 validated image filenames belonging to 3 classes.


In [13]:
checkpoint = ModelCheckpoint("best_model0.hdf5", monitor='val_acc', verbose=1,
    save_best_only=True, mode='auto', period=1)

In [52]:
#training model
model = Model_gender_test2()
history = model.fit_generator(train_generator,
                              steps_per_epoch=50,
                              epochs=70,
                              validation_data = validation_generator,
                              validation_steps=50,
                              callbacks=[checkpoint])
model.save('face_gender_detection.h5')

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
batch_normalization_96 (Batc (None, 148, 148, 32)      128       
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
batch_normalization_97 (Batc (None, 74, 74, 32)        128       
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 72, 72, 32)        9248      
_________________________________________________________________
batch_normalization_98 (Batc (None, 72, 72, 32)        128       
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 70, 70, 64)       

50/50 [==============================] - ETA: 0s - loss: 0.7904 - acc: 0.6540
Epoch 00046: val_acc did not improve from 0.69600
50/50 [==============================] - 82s 2s/step - loss: 0.7904 - acc: 0.6540 - val_loss: 0.7769 - val_acc: 0.6960
Epoch 47/70
50/50 [==============================] - ETA: 0s - loss: 0.9069 - acc: 0.6000
Epoch 00047: val_acc did not improve from 0.69600
50/50 [==============================] - 82s 2s/step - loss: 0.9069 - acc: 0.6000 - val_loss: 1.1194 - val_acc: 0.5293
Epoch 48/70
50/50 [==============================] - ETA: 0s - loss: 0.9109 - acc: 0.6020
Epoch 00048: val_acc did not improve from 0.69600
50/50 [==============================] - 82s 2s/step - loss: 0.9109 - acc: 0.6020 - val_loss: 0.8921 - val_acc: 0.6460
Epoch 49/70
50/50 [==============================] - ETA: 0s - loss: 0.8289 - acc: 0.6700
Epoch 00049: val_acc did not improve from 0.69600
50/50 [==============================] - 82s 2s/step - loss: 0.8289 - acc: 0.6700 - val_loss: 0

In [ ]:
Y_pred = model.predict_generator(test_generator, (9296 / 20)) # ceil(num_of_test_samples / batch_size)
target_names = ['Men', 'Woman','Baby']
print(classification_report(test_generator.classes[:Y_pred.shape[0]],
                            list(np.argmax(Y_pred,axis=1)), 
                            target_names=target_names))


acc_train = history.history['acc']
acc_val = history.history['val_acc']
epochs = range(1,51)
plt.plot(epochs, acc_train, 'g', label='Training accuracy')
plt.plot(epochs, acc_val, 'b', label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(1,51)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Age Prediction

In [29]:
def Model_age():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape = (150, 150, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(64, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(128, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(256, (1, 1), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(512, (1, 1), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Flatten())
    model.add(Dense(units = 192, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units = 128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units = 32, activation = 'relu'))
    model.add(Dense(units = 12, activation = 'softmax'))
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer = optimizers.RMSprop(lr=1e-4), metrics=['acc'])
    
    return model

In [20]:
train_images_ids.age.replace(['35','(8, 12)','13','22','34','45','(27, 32)','23','55','36','(38, 42)','57'],
                             ['(33,37)','(8,13)','(8,13)','(21,24)','(33,37)','(44,47)','(25, 32)','(21,24)','(54,59)',
                             '(33,37)','(38, 43)','(54,59)'],
                             inplace = True)


In [21]:
train_images_ids.age.value_counts()

(25, 32)     2528
(0, 2)       1255
(8,13)       1171
(38, 43)     1152
(4, 6)       1082
(15, 20)      804
(60, 100)     444
(48, 53)      381
(33,37)       225
(21,24)       120
(44,47)        44
(54,59)        44
None           20
3               7
29              7
58              4
(38, 48)        3
2               3
42              1
46              1
Name: age, dtype: int64

In [22]:
train_images_ids.drop(train_images_ids[~train_images_ids.age.isin((train_images_ids.age.value_counts().head(12)).index)].index, 
                      inplace = True)

In [23]:
train_images_ids.age.value_counts()

(25, 32)     2528
(0, 2)       1255
(8,13)       1171
(38, 43)     1152
(4, 6)       1082
(15, 20)      804
(60, 100)     444
(48, 53)      381
(33,37)       225
(21,24)       120
(44,47)        44
(54,59)        44
Name: age, dtype: int64

In [24]:
test_images_ids.age.replace(['35','(8, 12)','13','22','34','45','(27, 32)','23','55','36','(38, 42)','57'],
                             ['(33,37)','(8,13)','(8,13)','(21,24)','(33,37)','(44,47)','(25, 32)','(21,24)','(54,59)',
                             '(33,37)','(38, 43)','(54,59)'],
                             inplace = True)
test_images_ids.age.value_counts()

(25, 32)     2502
(0, 2)       1233
(38, 43)     1187
(8,13)       1116
(4, 6)       1058
(15, 20)      838
(48, 53)      444
(60, 100)     423
(33,37)       229
(21,24)       125
(54,59)        56
(44,47)        44
None           20
3              11
29              4
(38, 48)        3
58              1
(8, 23)         1
Name: age, dtype: int64

In [25]:
test_images_ids.drop(test_images_ids[test_images_ids.age.isin((test_images_ids.age.value_counts().loc[(test_images_ids.age.value_counts().values < 30)]).index)].index, 
                     inplace = True)

In [26]:
test_images_ids.age.value_counts()

(25, 32)     2502
(0, 2)       1233
(38, 43)     1187
(8,13)       1116
(4, 6)       1058
(15, 20)      838
(48, 53)      444
(60, 100)     423
(33,37)       229
(21,24)       125
(54,59)        56
(44,47)        44
Name: age, dtype: int64

In [50]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
                    train_images_ids,
                    x_col='original_image',
                    y_col='age',
                    directory=train_faces,
                    target_size=(150,150),
                    batch_size=100,
                    class_mode='categorical'
                )

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
                    test_images_ids,
                    x_col='original_image',
                    y_col='age',
                    directory=test_faces,
                    target_size=(150,150),
                    batch_size=100,
                    class_mode='categorical')

Found 9250 validated image filenames belonging to 12 classes.
Found 9255 validated image filenames belonging to 12 classes.


In [51]:
model_age = Model_age()
history_age = model_age.fit(train_generator,
                              steps_per_epoch=93,
                              epochs=50,
                              validation_data = test_generator,
                              validation_steps=93)
model_age.save('face_age_detection.h5')

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_78 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_76 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_79 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_77 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_78 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_81 (Conv2D)           (None, 17, 17, 256)     

KeyboardInterrupt: 